In [11]:
import pandas as pd

ods = (
    pd.read_excel(
        "../DB AF Completo 1.ods",
    )
    .loc[:, ["ID", "Phenotype", "Low Voltage / Scar area", "Rhythm during Mapping"]]
    .rename(
        columns={
            "ID": "Patient ID",
            "Low Voltage / Scar area": "Substrate",
            "Rhythm during Mapping": "Rhythm",
        }
    )
    .dropna(axis=0, how="any")
    .astype({"Patient ID": int})
)
csv = (
    pd.read_csv("thrs_<20_no_filt/lase.csv", sep=",")
    .dropna(axis=0, how="any")
    .astype({"Patient ID": int})
)
display(ods)
csv

,Patient ID,Phenotype,Substrate,Rhythm
3,9,Persistent,Abnormal,AF
4,10,Paroxysmal,Normal,SR
5,11,Persistent,Normal,AF
6,12,Persistent,Abnormal,SR
7,13,Persistent,Abnormal,AF
...,...,...,...,...
171,154,Parossistica,Normale,RS
177,160,Parossistica,Normale,RS
178,161,Parossistica,Normale,RS
179,162,Parossistica,Normale,RS


,Patient ID,Entropy,LASE
3,100,0,2.159463
4,101,0,1.504809
5,102,0,1.210491
6,103,0,1.965870
7,104,0,2.112994
...,...,...,...
95,94,0,2.088667
96,95,0,1.477727
97,96,0,2.058936
98,97,0,1.947283


In [12]:
# Join the two dataframes on the "ID" column
merged_df = pd.merge(
    ods.loc[:, ["Patient ID", "Phenotype", "Substrate", "Rhythm"]],
    csv,
    on="Patient ID",
    how="inner",
)

phenotype_mapping = {
    "Long standing persistent": "Persistente",
    "Persistente": "Persistente",
    "Parossistica": "Parossistica",
    "Parossistico": "Parossistica",
}

substrate_mapping = {
    "Normale": "Normale",
    "Abnormal": "Anormale",
    "Patologia": "Anormale",
    "Patologia ": "Anormale",
    "Patologia altri distretti": "Anormale",
    "Patologia parete posteriore": "Anormale",
    "Patologia parete posteriore e anteriore": "Anormale",
    "Patologia posteriore + altri": "Anormale",
    "Patologia posteriore e+ altri": "Anormale",
}

rhythm_mapping = {
    "AF": "FA",
    "FA": "FA",
    "SR": "RS",
    "RS": "RS",
}

merged_df["Phenotype"] = merged_df["Phenotype"].map(phenotype_mapping)
merged_df["Substrate"] = merged_df["Substrate"].map(substrate_mapping)
merged_df["Rhythm"] = merged_df["Rhythm"].map(rhythm_mapping)

merged_df

,Patient ID,Phenotype,Substrate,Rhythm,Entropy,LASE
0,40,Parossistica,Anormale,RS,0,1.898569
1,41,Parossistica,Normale,RS,0,2.137667
2,42,Parossistica,Normale,RS,0,1.422259
3,43,Persistente,Anormale,FA,0,1.346224
4,45,Persistente,Anormale,FA,0,1.632895
...,...,...,...,...,...,...
87,147,Parossistica,Normale,RS,0,1.933349
88,148,Parossistica,Normale,RS,0,1.791315
89,149,Parossistica,Normale,RS,0,1.300020
90,150,Parossistica,Normale,RS,0,2.284032


In [13]:
x = merged_df.loc[:, ["Phenotype"]]
y = merged_df.loc[:, ["Rhythm"]]

confusion_matrix = pd.crosstab(
    x["Phenotype"],
    y["Rhythm"],
    rownames=["Phenotype"],
    colnames=["Rhythm"],
    margins=True,
)
confusion_matrix = confusion_matrix.rename(
    index={"All": "Totale"}, columns={"All": "Totale"}
)
confusion_matrix

Rhythm,FA,RS,Totale
Phenotype,,,
Parossistica,3,53,56
Persistente,31,0,31
Totale,34,53,87


In [22]:
merged_df[
    (merged_df.loc[:, "Phenotype"] == "Parossistica")
    & (merged_df.loc[:, "Rhythm"] == "FA")
]

,Patient ID,Phenotype,Substrate,Rhythm,Entropy,LASE
55,100,Parossistica,Normale,FA,0,2.159463
84,144,Parossistica,Normale,FA,0,2.023048
85,145,Parossistica,Normale,FA,0,1.655584
